In [88]:
import xeofs

# Import the derived EOF
eof = xeofs.single.EOF.load("/cluster/home/haroldh/spGDMM/1_data/4_interim/EOFs/EOF_tsuv.nc")

features = eof.components() * eof.scores()
features = features.sum(dim="variable")
del features.attrs['solver_kwargs']
features

<xarray.DataArray (mode: 8, yc: 555, xc: 950, time: 365)> Size: 12GB
dask.array<sum-aggregate, shape=(8, 555, 950, 365), dtype=float64, chunksize=(1, 111, 950, 365), chunktype=numpy.ndarray>
Coordinates:
  * yc       (yc) float32 2kB 800.0 1.6e+03 2.4e+03 ... 4.432e+05 4.44e+05
  * xc       (xc) float32 4kB 800.0 1.6e+03 2.4e+03 ... 7.592e+05 7.6e+05
  * mode     (mode) int64 64B 1 2 3 4 5 6 7 8
  * time     (time) int64 3kB 0 1 2 3 4 5 6 7 ... 358 359 360 361 362 363 364
Attributes: (12/14)
    center:        True
    check_nans:    True
    compute:       True
    date:          2025-01-15 16:32:52
    feature_name:  feature
    model:         EOF analysis
    ...            ...
    sample_name:   sample
    software:      xeofs
    solver:        auto
    standardize:   True
    use_coslat:    False
    version:       3.0.4

In [91]:
features.to_netcdf("/cluster/home/haroldh/spGDMM/1_data/4_interim/EOF_features_with_time.nc", mode="w")

: 

In [30]:
features_averaged = features.mean(dim="time")
features_averaged

<xarray.DataArray (mode: 3, yc: 555, xc: 950)> Size: 13MB
dask.array<mean_agg-aggregate, shape=(3, 555, 950), dtype=float64, chunksize=(1, 111, 950), chunktype=numpy.ndarray>
Coordinates:
  * yc       (yc) float32 2kB 800.0 1.6e+03 2.4e+03 ... 4.432e+05 4.44e+05
  * xc       (xc) float32 4kB 800.0 1.6e+03 2.4e+03 ... 7.592e+05 7.6e+05
  * mode     (mode) int64 24B 1 2 3
Attributes: (12/14)
    center:        True
    check_nans:    True
    compute:       True
    date:          2025-01-15 16:32:52
    feature_name:  feature
    model:         EOF analysis
    ...            ...
    sample_name:   sample
    software:      xeofs
    solver:        auto
    standardize:   True
    use_coslat:    False
    version:       3.0.4

In [35]:
import sys

sys.path.append('/cluster/home/haroldh/spGDMM/3_src/1_data')
stat
from fetch_sinmod_data import fetch_sinmod_data
from create_sample_locations import create_sample_locations

no_sites = 100
time_idx = 285
# Creating biomod training data
sampled_locations = create_sample_locations(no_sites, time_idx)

sampled_locations

,x,y,time_idx
0,110400.0,284800.0,285
1,524000.0,398400.0,285
2,751200.0,286400.0,285
3,709600.0,257600.0,285
4,49600.0,201600.0,285
...,...,...,...
95,505600.0,336800.0,285
96,649600.0,415200.0,285
97,722400.0,225600.0,285
98,622400.0,287200.0,285


In [6]:
import xarray as xr
import pandas as pd

def fetch_data(data_path, sampled_locations, type, no_modes=None, env_variables=None, normalise=False):
    """
    Fetch EOF data for specific (x, y, time) combinations defined in sampled_locations.

    Parameters:
        data_path (str): Path to the EOF NetCDF file.
        sampled_locations (pd.DataFrame): DataFrame with columns ['x', 'y', 'time_idx'] for sample locations.
        type (str): Type of data to fetch ('target', 'stat', 'EOF_time').
        no_modes (int, optional): Number of EOF modes to fetch. Required if type is 'EOF_time'.
        env_variables (list, optional): List of environmental variable names to extract. Required if type is 'target' or 'stat'.
        normalise (bool, optional): Whether to normalise the fetched data. Only applicable if type is 'target'.

    Returns:
        pd.DataFrame: DataFrame with EOF data for the specified variables and locations.
    """

    if no_modes and env_variables:
        raise ValueError("Cannot specify both 'no_modes' and 'env_variables'. Please choose one.")

    # Open the SINMOD NetCDF file
    data = xr.open_dataset(data_path)
    
    # Prepare the result DataFrame by copying sampled_locations
    result = sampled_locations.copy()

    # Iterate over each environmental variable
    if env_variables:
        for var in env_variables:
            
            if type == 'target':
                # Fetch data for each row in sampled_locations
                result[var] = sampled_locations.apply(
                    lambda row: data[var].isel(time=int(row['time_idx'])).sel(
                        xc=row['x'], 
                        yc=row['y'], 
                        method="nearest"
                    ).values.item(),
                    axis=1
                )
            elif type == 'stat':
                # Iterate over each `stat` dimension for the variable
                for stat in data[var].stat.values:
                    # Fetch data for each stat dimension and label column as `{var}_{stat}`
                    result[f"{var}_{stat}"] = sampled_locations.apply(
                        lambda row: data[var].sel(stat=stat).sel(xc=row['x'], yc=row['y'], method="nearest").values.item(),
                        axis=1
                    )
    
    elif no_modes:
        if type == 'EOF_time':
            
            data = data.to_dataarray()

            if no_modes > data.mode.values.max():
                raise ValueError("Number of modes to keep exceeds the maximum number of modes in the data")

            # Iterate over each `mode` dimension for the variable
            for mode in range(1, no_modes + 1):
                # Fetch data for each mode dimension and label column as `{var}_{stat}`
                result[f"Mode_{mode}"] = sampled_locations.apply(
                    lambda row: data.sel(mode=mode, xc=row['x'], yc=row['y'], method="nearest").isel(time=int(row['time_idx'])).values.item(),
                    axis=1
                )
        else:
            raise ValueError("Check type of data to fetch")
    # Close the dataset
    data.close()

    if type == 'target':
        result[env_variables] = result[env_variables].fillna(0)
        if normalise:
            result[env_variables] = result[env_variables].div(result[env_variables].sum(axis=1), axis=0)
            
    return result


In [86]:
data = fetch_EOF_data("/cluster/home/haroldh/spGDMM/1_data/4_interim/EOF_features_with_time.nc", sampled_locations, type='EOF_time')

In [15]:
import sys

sys.path.append('/cluster/home/haroldh/spGDMM/3_src/1_data')

from create_sample_locations import create_sample_locations

no_sites = 100
time_idx = 285
# Creating biomod training data
sampled_locations = create_sample_locations(no_sites, time_idx)

# Define constants
biomod_sinmod_path = "/cluster/home/haroldh/spGDMM/1_data/1_raw/biostates_surface_normalised.nc"
biomod_target_variables = ['diatoms', 'flagellates', 'ciliates', 'HNANO', 'bacteria', 'calanus_finmarchicus', 'calanus_glacialis','detritus_slow', 'detritus_fast',
                    'nitrate','silicate','ammonium','DOC','cDOM', 'silicate_detritus','sediment_Si','sediment_N']

biomod = fetch_data(biomod_sinmod_path, sampled_locations, type='target', env_variables = biomod_target_variables)

# Define constants
# Here the SINMOD_features dataset must have a 'stat' dimension. ALl features in stat dimension will be used.
path = '/cluster/home/haroldh/spGDMM/1_data/4_interim/SINMOD_features.nc'
target_variables = [
    'temperature', 'salinity'
]

midnor = fetch_data(path, sampled_locations, type='stat', env_variables = target_variables)

In [16]:
biomod

,x,y,time_idx,diatoms,flagellates,ciliates,HNANO,bacteria,calanus_finmarchicus,calanus_glacialis,detritus_slow,detritus_fast,nitrate,silicate,ammonium,DOC,cDOM,silicate_detritus,sediment_Si,sediment_N
0,110400.0,284800.0,285,0.133621,0.006626,0.344889,0.116769,0.089071,0.001083,0.0,0.193278,0.000864,0.203214,0.345650,0.160506,0.020725,0.0,0.000167,0.283319,0.006546
1,524000.0,398400.0,285,0.071390,0.006511,0.238889,0.089497,0.040332,0.053102,0.0,0.175446,0.000864,0.686482,0.731525,0.323443,0.007484,0.0,0.001416,0.417737,0.011353
2,751200.0,286400.0,285,0.111261,0.004798,0.289111,0.117718,0.072758,0.089358,0.0,0.226481,0.006050,0.238221,0.412068,0.252918,0.006333,0.0,0.001166,0.277779,0.022950
3,709600.0,257600.0,285,0.112473,0.004684,0.336889,0.114180,0.062950,0.023316,0.0,0.231810,0.000000,0.300088,0.413003,0.261674,0.011514,0.0,0.000666,0.257515,0.013642
4,49600.0,201600.0,285,0.140625,0.005369,0.376223,0.116165,0.079764,0.086184,0.0,0.246977,0.000864,0.345883,0.417680,0.258269,0.022453,0.0,0.001333,0.303293,0.012894
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,505600.0,336800.0,285,0.113685,0.005141,0.318444,0.111850,0.056545,0.042680,0.0,0.211519,0.000000,0.361735,0.484098,0.267024,0.017271,0.0,0.000750,0.219733,0.008316
96,649600.0,415200.0,285,0.101832,0.007083,0.220889,0.105463,0.061649,0.048875,0.0,0.170117,0.000864,0.523118,0.613658,0.296206,0.012665,0.0,0.000999,0.375561,0.007813
97,722400.0,225600.0,285,0.122575,0.006169,0.363778,0.109605,0.079764,0.003525,0.0,0.186718,0.001729,0.254513,0.399438,0.160992,0.014392,0.0,0.000999,0.274957,0.020967
98,622400.0,287200.0,285,0.095097,0.006397,0.234889,0.097350,0.057746,0.113849,0.0,0.179750,0.000864,0.472259,0.551917,0.207685,0.013817,0.0,0.001915,0.230445,0.021043


In [12]:
midnor

,x,y,time_idx,temperature_mean,temperature_10th_percentile,temperature_90th_percentile,salinity_mean,salinity_10th_percentile,salinity_90th_percentile
0,110400.0,284800.0,285,10.173243,8.722788,12.260446,34.943367,33.975800,35.394604
1,524000.0,398400.0,285,9.224305,7.700943,11.483626,35.235497,35.066376,35.333569
2,751200.0,286400.0,285,9.113077,7.145188,11.988599,34.510536,33.973053,34.952545
3,709600.0,257600.0,285,9.006870,7.381804,11.758172,34.759804,34.455551,35.062866
4,49600.0,201600.0,285,9.924729,8.138945,12.194427,34.651325,34.012573,35.195930
...,...,...,...,...,...,...,...,...,...
95,505600.0,336800.0,285,9.622821,8.238765,11.800544,35.079910,34.656971,35.311440
96,649600.0,415200.0,285,9.131057,7.677932,11.049117,35.254898,35.181282,35.317543
97,722400.0,225600.0,285,9.173612,7.269289,11.931150,34.430244,33.755150,34.992828
98,622400.0,287200.0,285,9.234761,7.676504,11.983837,34.933239,34.684898,35.168308
